Tutorial 3 - Dynamic Flowsheets
=============================

Introduction
------------

The previous tutorials have looked at developing flowsheets for steady-state processes, however it is often important to study how a process behaves under transient conditions. This tutorial will demonstrate how to create a model for a dynamic process using the IDAES modleing framework.

For this tutorial, we will use a similar process to the previous tutorials, using two CSTRs to perform a simple chemicl reaction. However, for this tutorial we will add a second feed stream and a mixer unit before hte first reactor, to represent the mixing of two reactant streams as shown below.

<img src="mix_2cstrs.png">

<center>ethyl acetate + NaOH $\rightarrow$ sodium acetate + ethanol</center>

In this tutorial, you will learn how to:

* stup a dynamic flowsheet and define the time domain
* add additonal variables and constraints to a Unit Model
* transform a time domain to automatically populate the time domain and define derivative terms
* set initial conditions for a dynamic model
* initialize and solve a dynamic model
* introduce a step change to the model
* plot profiles of key variables as a function of time

Importing Libraries
----------------------------

As before, the first step in creating our model is to import the necessary components from Python, Pyomo and IDAES. FThe componets we will require are discussed below.

Firstly, we need all the components from Pyomo that we have used in the previous tutorials (`ConcreteModel`, `SolverFactory` and  `TransformationFactory` from `pyomo.environ` and `Arc` from `pyomo.network`). In addition to this, we are going to the need the `Constraint` and `Var` components from `pyomo.environ` in order to add additonal variables and constraints to our model.

Import these components below:

In [ ]:
from pyomo.environ import (ConcreteModel, SolverFactory, TransformationFactory,
                           Var, Constraint)
from pyomo.network import Arc

Next, we will need the IDAES compoennts we have used in the previous tutorials:

`FlowsheetBlock` from `idaes.core`, 
`idaes.property_models.examples.saponification_thermo`
`idaes.property_models.examples.saponification_reactions`
`CSTR` from `idaes.unit_models`

We will also need the `Mixer` model from the IDAES unit model library (`idaes.unit_models`)

Import these components below:

In [ ]:
from idaes.core import FlowsheetBlock

import idaes.property_models.examples.saponification_thermo as thermo_props
import idaes.property_models.examples.saponification_reactions as \
    reaction_props

from idaes.unit_models import CSTR, Mixer

Finally, we are going to need a tool for plotting our results. For this tutorial, we will use the `pyplot` tool in `matplotlib` (part of most scientific Python distributions), which we will import as `plt` for convenience.

If you do not have `matplotlib` installed in your Python distribution, see you Python package manager for details on how to install it.

The code to import `pyplot` is given below.

In [ ]:
import matplotlib.pyplot as plt

Creating Our Model
-----------------------------

Now that we have imported all the tools we will need, we can begin constructing our model. As before, the first step is to create a `ConcreteModel` to hold our flowhseet.

In [ ]:
m = ConcreteModel()

Next, we add a `FlowsheetBlock` to our model. In previous tutorials, we have set the `dynamic` argument to be `False`, indicating a steady-state flowsheet. In this case however, we wish to create a dynamic flowsheet, so we set this argumnet to `True`.

Additionally, for a dynamic model, we need to provide some information about the time domain for our model. At a minimum, we need to set the start and end times for our time domain, which for this case will be 0 and 8 seconds respectively. We can also specify specific time points that we wish to ensure are in our time domain, such as points where we know a step-chenge will occur. For this tutorial, we will introduce a step-change at t = 1 second later in the tutorial, so we will include this in our time domain now.

Thus, we want to create a time domain that begins at t = 0, has a point at t = 1 and ends at t = 10. To do this, we use the `time_set` argument when creating our `FlowhseetBlock` and provide these values as a Python list (or equivalent) - i.e. `time_set: [0, 1, 10]` (note that the time vlaues may be integers or floating point numbers at this stage).

The code to do this is shown below:

In [ ]:
m.fs = FlowsheetBlock(default={"dynamic": True, "time_set": [0, 1, 10]})

To see what happened in this set, let us `display` the time domain of our flowhseet.

In [ ]:
m.fs.time.display()

We should see the following:

```
time : Dim=0, Dimen=1, Size=3, Domain=None, Ordered=Sorted, Bounds=(0.0, 10.0)
    [0.0, 1.0, 10.0]
```

This shows us that our `time` domain is a sorted Pyomo `Set` (`Ordered=Sorted`) with points at 0.0, 1.0 and 10.0 and bounds of 0.0 and 10.0. This is waht we would expect, as we told the `FlowsheetBlock` to create a time domain with these points.

The fact that the `Set` is sorted is important, as it indicates the points in the `Set` are in order, allowing us to find the next and previous points if required.

Next we need to add the property parameter blocks to our flowsheet, which is done in the same way as for steady-state flowsheets. Remember to link the `ReactionParameterBlock` to the `PhysicalParameterBlock`.

In [ ]:
m.fs.thermo_params = thermo_props.SaponificationParameterBlock()
m.fs.reaction_params = reaction_props.SaponificationReactionParameterBlock(
    default={"property_package": m.fs.thermo_params})

Just as for steady-state models, the next step is to add the necessary unit models to our flowsheet. As we have declared our flowsheet to be a dynamic flowsheet, all unit models within it will automatically be assumed to be dynamic unit operations. However, there are some cases where we might want to placea steady-state model within an otherwise dynamic flowsheet (e.g. for a unit with a very fast response time relative to the rest of the process). The IDAES framework supports this by allowing us to specifically state that a unit operation is steady-state, overriding the automatic assumption of a dynamic unit. Note that the oppositie is not true, and that we cannot place dynamic models within a steady-state flowsheet.

In our flowsheet, the first unit operation is a `Mixer` unit. Normally we assume an indeal mixer, which has zero volume and thus no holdup or accumulation - i.e. it is effectively a steady-state unit operation (in more rigorous models we might use a tank model to represent a real mixer). Thus, let us add a `Mixer` unit to our flowsheet and declare it to be a steady-state unit operation. To do this, we simply pass the following argumnet to the unit model as we construct it: `"dynamic": False` to indicate that the model should be a steady-state model. A `Mixer` unit also requires a physical property package (but not a reaction package) to define the list of components that are present in the unit, so we also need to pass the `"property_package"` argument pointing to the appropriate `PhysicalParameterBlock`.

In [ ]:
m.fs.mix = Mixer(default={"dynamic": False,
                          "property_package": m.fs.thermo_params})

Next, we need to add the two `CSTR` units to our flowhseet, which is done in the same way as in the previous tutorials.

In [ ]:
m.fs.Tank1 = CSTR(default={"property_package": m.fs.thermo_params,
                           "reaction_package": m.fs.reaction_params,
                           "has_holdup": True,
                           "has_equilibrium_reactions": False,
                           "has_heat_transfer": True,
                           "has_pressure_change": False})
m.fs.Tank2 = CSTR(default={"property_package": m.fs.thermo_params,
                           "reaction_package": m.fs.reaction_params,
                           "has_holdup": True,
                           "has_equilibrium_reactions": False,
                           "has_heat_transfer": True,
                           "has_pressure_change": False})

Whilst the unit models alone are sufficient for a steady-state flowsheet, dynamic flowsheets require extra constraints to determine the flow of material in the process. In most cases, we need to write a set of pressure-flow constraints for each dynamic unit in the process to determine the rate at which material leaves the unit.

For this tutorial, we will write a set of simple constraints for each `CSTR` with the form:

<center>$F = C \times h$</center>

where $F$ is volumetric flowrate, $C$ is a flow coeffiicent and $h$ is the depth of fluid within the CSTR. Additionally, we will need a constraint that relates depth to the volume of the tank:

<center>$V = A \times h$</center>

where $V$ is volume and $A$ is the cross-section area of the tank.

To do this, first we need to create some new variables in our `CSTR` units. The `CSTRs` already have variables for the volume and the volumetric flowrate, so we need to create new variables for the flow coefficient, depth and area. To do this, we create new Pyomo `Var` objects in each `CSTR`. To create a `Var` for area, we do the following:

In [ ]:
m.fs.Tank1.area = Var(initialize=1.0,
                      doc="Cross-sectional area of tank [m^2]")

Here, we are creating a new `Var` object named `area` within the first CSTR (`m.fs.Tank1`). We have also given it an initial value of 1.0 using the `initialize` argument, and a dcoumentation string to explain what the variable is (thorugh the `doc` argument).

In the case of `area`, we know that it will not vary with time and thus we created a single `Var` to represent the area at all points in time. However, we expect the depth and potentially the flow coefficient to vary with time as the amount of fluid in the reactor and the valve opening changes. For these variables, we need to create `Var` objects which are indexed by `time`. In Pyomo (and thus IDAES), this is done by providing the indexing set (or indexing sets) as arguments when creating the `Var` object (or any other component).

For example, we create the flow coeffiicent as shown below:

In [ ]:
m.fs.Tank1.flow_coeff = Var(m.fs.time,
                            initialize=5e-5,
                            doc="Tank outlet flow coefficient")

Now try creating a `Var` named `height` below to represent the depth of fluid in the reactor.

In [ ]:
m.fs.Tank1.height = Var(m.fs.time,
                        initialize=1.0,
                        doc="Depth of fluid in tank [m]")

Now that we have the variables we need to represent the pressure driven flow, we need to write the `Constraints` which relate these. Let us start wit hthe simpler of the two constraints, namely $V = A \times h$.

To begin wtih, each of the `Constraints` we are going to construct need to be indexed by `time`, as they involve `Vars` that are indexed by time. To do this, we need to create a method or rule which tells Pyomo how to construct the `Constraint`. For this, we declare a method using the Python `def` keyword followed by a name for the method (which we will call `geometry`). This is followed by a pair of parenthesis which contain the following arguments for our method:

* the first argument is a placeholder for the model on which the `Constraint` will be constructed. This name can be anything you want (as it is just a placeholder), but within IDAES we tend to use either b (for "block") or m (for "model).
* placeholders for each index that will be used in the `Constraint`. In our case our only index is `time` (which we will represent with t), but there can be as many indexes as required.

The result is the following:

```
def geometry(b, t):
```

Note that the line of code ends with a `:`.

Next, we need to add the body of our method, which needs to return an *expression* that describes the `Constraint` to be written at each combination of indices. More complex `Constraints` may involve `if/else` statements here, but in our case we wish to have the same form of the expression at all points in time, namely $V_t = A \times h_t$. To acheive this, we write the following:

```
    return b.volume[t] == b.area*b.height[t]
```

Firstly, note that this line of code is indented - this is Python notatation to show that this line of code is part of the previously declared method. Next, the line begins with a `return` command, which indicates that the method will return the result of this line to whatever called it (which will be the `Constraint` constructor in our case). Finally, the rest of the line is the *expression* that we wish to return. You will see that we have used our placeholder for the model (`b`) when referencing the `Vars` in that model, and that we have included time indices on the appropriate `Vars` (using `[t]`).

Lastly, we need to add the actual `Constraint` to our model. Similar to adding a `Var` to our CSTRs, we now add a `Constraint` to the CSTR by declaring a `Constraint` with the name `geometry` and passing the following arguments to it:

* the first index (or indices) are the indexing sets for the `Constraint`
* we set the rule for the `Constraint` using the `rule` argument, and point it to the method we just wrote.

```
m.fs.Tank1.geometry = Constraint(m.fs.time, rule=geometry)
```

The completed code is shown below for you to run.

In [ ]:
def geometry(b, t):
    return b.volume[t] == b.area*b.height[t]
m.fs.Tank1.geometry = Constraint(m.fs.time, rule=geometry)

Next, we need to add the `Constraint` for $F_t = C_t \times h_t$. We use the same steps as above, but in this case we have one minor complication - what is the name of the variable for $F_t$? We created `Vars` for $C$ and $h$ ourselves, but $F$ is constructed as part of the `CSTR` model, so we need to go and find its name.

To do this, we need to understand something about the structure of the `CSTR` model (and IDAES models in general). Unit models within IDAES (or at least the core model libraries) use a standardized structure based on the concept of *control volumes*. Each unit model contains one or more control volumes, which represent distinct volumes of fluid over which material, energy and momentum balances will be written. Each control volume then contains a number of *state blocks* which contain the information on the state of the material at a specific time and location within the control volume (both extensive and intensive).

The structure of unit models will be discussed more in later tutorials, but for now it is sufficient to know that our `CSTR` contains a single control volume (names `control_volume`), with two state blocks named `properties_in` and `properties_out` which are indexed by time.

So, we want the volumetric flowrate of material leaving the CSTR. The IDAES documentation `[add reference]` contains a list of standard names for common properties, which shows us that volumetric flowrate should be named `flow_vol`, so the variable we want (at time $t$) should be called:

```
tank.control_volume.properties_out[t].flow_vol
```

Thus, the code to construct of pressure driven flow constraint is:

In [ ]:
def outlet_flowrate(b, t):
    return b.control_volume.properties_out[t].flow_vol == b.flow_coeff[t]*b.height[t]
m.fs.Tank1.outlet_flowrate = Constraint(m.fs.time, rule=outlet_flowrate)

We now need to do the same procedure for `Tank2`. First, try creating the three `Vars` (`area`, `height` and `flow_coeff`) required for the `Constraints` in `Tank2` (don't forget to change the name of the tank if you copy from the code above).

In [ ]:
m.fs.Tank2.height = Var(m.fs.time,
                        initialize=1.0,
                        doc="Depth of fluid in tank [m]")
m.fs.Tank2.area = Var(initialize=1.0,
                      doc="Cross-sectional area of tank [m^2]")
m.fs.Tank2.flow_coeff = Var(m.fs.time,
                            initialize=5e-5,
                            doc="Tank outlet flow coefficient")

Next, we need to construct the `Constraints` for `Tank2`. One of the good things about the way Pyomo and Python work is that we can create general methods to do common tasks to prevent ourselves from having to repeat our code. If you look at the methods we wrote whilst creating the `Constraints` for `Tank1`, you might notice that the rules are in fact general - we used a placeholder `b` in place of specific unit names, so we can actually reuse the same methods for `Tank2`. All we need to do is create the `Constraint` objects on `Tank2` as we did before and pass the same rule:

In [ ]:
m.fs.Tank2.geometry = Constraint(m.fs.time, rule=geometry)
m.fs.Tank2.outlet_flowrate = Constraint(m.fs.time, rule=outlet_flowrate)

Connecting the Flowsheet
--------------------------------------

In [ ]:
m.fs.stream1 = Arc(source=m.fs.mix.outlet, destination=m.fs.Tank1.inlet)
m.fs.stream2 = Arc(source=m.fs.Tank1.outlet, destination=m.fs.Tank2.inlet)

In [ ]:
m.discretizer = TransformationFactory('dae.finite_difference')
m.discretizer.apply_to(m,
                       nfe=200,
                       wrt=m.fs.time,
                       scheme="BACKWARD")

In [ ]:
TransformationFactory("network.expand_arcs").apply_to(m)

In [ ]:
m.fs.mix.inlet_1.flow_vol.fix(0.5)
m.fs.mix.inlet_1.conc_mol_comp[:, "H2O"].fix(55388.0)
m.fs.mix.inlet_1.conc_mol_comp[:, "NaOH"].fix(100.0)
m.fs.mix.inlet_1.conc_mol_comp[:, "EthylAcetate"].fix(0.0)
m.fs.mix.inlet_1.conc_mol_comp[:, "SodiumAcetate"].fix(0.0)
m.fs.mix.inlet_1.conc_mol_comp[:, "Ethanol"].fix(0.0)
m.fs.mix.inlet_1.temperature.fix(303.15)
m.fs.mix.inlet_1.pressure.fix(101325.0)

m.fs.mix.inlet_2.flow_vol.fix(0.5)
m.fs.mix.inlet_2.conc_mol_comp[:, "H2O"].fix(55388.0)
m.fs.mix.inlet_2.conc_mol_comp[:, "NaOH"].fix(0.0)
m.fs.mix.inlet_2.conc_mol_comp[:, "EthylAcetate"].fix(100.0)
m.fs.mix.inlet_2.conc_mol_comp[:, "SodiumAcetate"].fix(0.0)
m.fs.mix.inlet_2.conc_mol_comp[:, "Ethanol"].fix(0.0)
m.fs.mix.inlet_2.temperature.fix(303.15)
m.fs.mix.inlet_2.pressure.fix(101325.0)

In [ ]:
m.fs.Tank1.area.fix(1.0)
m.fs.Tank1.flow_coeff.fix(0.5)
m.fs.Tank1.heat_duty.fix(0.0)

m.fs.Tank2.area.fix(1.0)
m.fs.Tank2.flow_coeff.fix(0.5)
m.fs.Tank2.heat_duty.fix(0.0)

In [ ]:
m.fs.fix_initial_conditions()

In [ ]:
m.fs.mix.initialize()

In [ ]:
m.fs.Tank1.initialize(state_args={
            "flow_vol": m.fs.mix.outlet.flow_vol[0].value,
            "conc_mol_comp": {"H2O": m.fs.mix.outlet.conc_mol_comp[0, "H2O"].value,
                              "NaOH": m.fs.mix.outlet.conc_mol_comp[0, "NaOH"].value,
                              "EthylAcetate": m.fs.mix.outlet.conc_mol_comp[0, "EthylAcetate"].value,
                              "SodiumAcetate": m.fs.mix.outlet.conc_mol_comp[0, "SodiumAcetate"].value,
                              "Ethanol": m.fs.mix.outlet.conc_mol_comp[0, "Ethanol"].value},
            "temperature": m.fs.mix.outlet.temperature[0].value,
            "pressure": m.fs.mix.outlet.pressure[0].value})

In [ ]:
m.fs.Tank2.initialize(state_args={
            "flow_vol": m.fs.Tank1.outlet.flow_vol[0].value,
            "conc_mol_comp": {"H2O": m.fs.Tank1.outlet.conc_mol_comp[0, "H2O"].value,
                              "NaOH": m.fs.Tank1.outlet.conc_mol_comp[0, "NaOH"].value,
                              "EthylAcetate": m.fs.Tank1.outlet.conc_mol_comp[0, "EthylAcetate"].value,
                              "SodiumAcetate": m.fs.Tank1.outlet.conc_mol_comp[0, "SodiumAcetate"].value,
                              "Ethanol": m.fs.Tank1.outlet.conc_mol_comp[0, "Ethanol"].value},
            "temperature": m.fs.Tank1.outlet.temperature[0].value,
            "pressure": m.fs.Tank1.outlet.pressure[0].value})

In [ ]:
solver = SolverFactory('ipopt')
results = solver.solve(m.fs)

In [ ]:
for t in m.fs.time:
    if t > 1.0:
        m.fs.mix.inlet_2.conc_mol_comp[t, "EthylAcetate"].fix(90.0)
results = solver.solve(m.fs)

In [ ]:
print(results)

In [ ]:
plt.figure("Tank 1 Outlet")
plt.plot(m.fs.time,
         list(m.fs.Tank1.outlet.conc_mol_comp[:, "NaOH"].value),
         label='NaOH')
plt.plot(m.fs.time,
         list(m.fs.Tank1.outlet.conc_mol_comp[:, "EthylAcetate"].value),
         label='EthylAcetate')
plt.plot(m.fs.time,
         list(m.fs.Tank1.outlet.conc_mol_comp[:, "SodiumAcetate"].value),
         label='SodiumAcetate')
plt.plot(m.fs.time,
         list(m.fs.Tank1.outlet.conc_mol_comp[:, "NaOH"].value),
         label='Ethanol')
plt.legend()
plt.grid()
plt.xlabel("Time [s]")
plt.ylabel("Concentration [mol/m^3]")

In [ ]:
plt.figure("Tank 2 Outlet")
plt.plot(m.fs.time,
         list(m.fs.Tank2.outlet.conc_mol_comp[:, "NaOH"].value),
         label='NaOH')
plt.plot(m.fs.time,
         list(m.fs.Tank2.outlet.conc_mol_comp[:, "EthylAcetate"].value),
         label='EthylAcetate')
plt.plot(m.fs.time,
         list(m.fs.Tank2.outlet.conc_mol_comp[:, "SodiumAcetate"].value),
         label='SodiumAcetate')
plt.plot(m.fs.time,
         list(m.fs.Tank2.outlet.conc_mol_comp[:, "NaOH"].value),
         label='Ethanol')
plt.legend()
plt.grid()
plt.xlabel("Time [s]")
plt.ylabel("Concentration [mol/m^3]")

Summary
--------------

Differences between steady-state an dynmaic flowsheets